<a href="https://colab.research.google.com/github/pherreragalvez/big_data_science_diploma/blob/main/Ejercicio_3_NLP_para_an%C3%A1lisis_de_emociones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 3 - Análisis de emociones en texto no-estructurado (NLP)

La idea es realizar un análisis de emociones desde comentarios en medios sociales, por medio de un modelo de clasificación supervisada, que entrega una de 3 posibles clases de emoción.

## Contexto: Análisis de Texto de Comentarios en Medios Sociales de diversos países (Chile, Arg, Mex, otros)

Este conjunto de datos generado en 2016 por [R:Solver](http://rsolver.com) y compartido parcialmente para este tipo de ejercicios, consiste en dos columnas: el texto del comentario, y una clasificación dentro de las tres alternativas o clases de emoción.

El gran objetivo final a resolver con este ejemplo, es analizar una porción de texto y lograr predecir la emoción correspondiente, dentro de 3 posibles clases:

*     Enojo
*     Sorpresa
*     Alegría

**CONSIDERACIÓN IMPORTANTE**

Se sabe que para lograr buenos resultados se requiere contar con muchos miles de ejemplos y en este ejercicio académico sólo se cuenta con pocos miles por cada clase. En la práctica se podrá apreciar que el desempeño general (accuracy) de los modelos es mediano-bajo (cerca del 50%). Esto, seguramente, podría mejorar recién al contar con varias decenas de miles de ejemplos adicionales a los actuales, lo que se explica por la gran diversidad de terminología encontrada en estos comentarios (son cerca de 20.000 términos o palabras diferentes en este corpus).

Por esta razón, el foco está en realizar análisis sobre estos desempeños y sacar conclusiones útiles, hasta cierto punto independiente de los *accuracies* correspondientes. Este ejercicio busca reforzar conceptos desde la perspectiva de quienes ya están adentrándose más en entender el funcionamiento de modelos de clasificación supervisada.

---

## Instrucciones Generales

Se deben contestar las preguntas que se indican en las secciones de "Preguntas", más adelante. Se puede recurrir a ejercicios de otras fuentes, así como al material de clases.

La entrega se realiza en forma de un informe en formato PDF, utilizando la plantilla de informe que está en http://dcc.rsolver.com/dcc/docs/InformeActividad.docx

El informe en formato PDF debe ser subido por sólo uno de los integrantes a la siguiente URL

http://aiker.ai/aiker/DocUpload.aspx (*)

(*) Si hay problemas en la carga, enviar el PDF a rsandova@ing.puc.cl y cc: ayudante@aiker.ai


## Paso 1: Instalar librerías de modelos de clasificación

In [ ]:
install.packages('e1071')
install.packages('caret')
install.packages('caTools')
install.packages('tm')
install.packages('rpart')
install.packages('SnowballC')
install.packages('nnet')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘proxy’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘future.apply’, ‘progressr’, ‘numDeriv’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘iterators’, ‘Rcpp’, ‘gower’, ‘hardhat’, ‘ipred’, ‘tidyselect’, ‘timeDate’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘bitops’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘NLP’, ‘slam’, ‘BH’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is un

## Paso 2: Carga de los datos

La siguiente celda de código carga los datos desde la URL de origen y luego muestra un encabezado con las primeras filas del dataset, para demostrar la disposición y ejemplos de los datos.

**Nótese que hay un desbalance entre los ejemplos de cada una de las 3 clases, pero NO se espera ni se pide modificar esto, lo cual queda a criterio de los alumnos experimentar con re-balancear las clases y ver su desempeño.**


In [ ]:
# Se declara la URL de dónde obtener los datos
theUrlMain <- "http://RAlize.RSolver.com/RAlize/data/small_emotion_sample_2016.csv"

# Se declaran los nombres de las columnas
columnas <- c("texto","emoción")

# Se cargan datos principales a una estructura (commentsdataset), asignando nombres de atributos a las columnas
comments.dataset.raw <- read.csv(file = theUrlMain, header = FALSE, sep = ";", col.names=columnas, skipNul = TRUE)

# Conteo de ejemplos de cada clase.
# Al ver su dimensión, se puede apreciar si están o no balanceados,
# pero no se espera en esta ocasión trabajar en balanceo de clases.
data.enojo <- comments.dataset.raw[comments.dataset.raw$emoción == 'enojo',]
data.sorpresa <- comments.dataset.raw[comments.dataset.raw$emoción == 'sorpresa',]
data.alegria <- comments.dataset.raw[comments.dataset.raw$emoción == 'alegria',]
cat("\nEjemplos Enojo:     ", dim(data.enojo))
cat("\nEjemplos Sorpresa:  ", dim(data.sorpresa))
cat("\nEjemplos Alegría:   ", dim(data.alegria))

commentsdataset <- comments.dataset.raw
dim(commentsdataset)
head(commentsdataset, 20)


Ejemplos Enojo:      3054 2
Ejemplos Sorpresa:   1085 2
Ejemplos Alegría:    2762 2

[1] 6901    2

,texto,emoción
,<chr>,<chr>
1,!FAVOR REVISAR DICCIONARIO !,enojo
2,?El Dodge Challenger cuantos kilometros por litro dara? Esa es mi consulta,sorpresa
3,"el trasero trailer de la version en carne y hueso , esos redactores merecen el premio redactor de excelencia jajaj",alegria
4,No sera un Peruano o Boliviano que lo hace para que sancionen a Chile ? Hay que acordarse de la guerra mediatica y todos los reclamos de parte de Evo A ese Evo tambien se le sospecha que fue quien dio la orden del reclamo boliviano por los canticos en el estadio,enojo
5,Y Renaca y Concon para levantar carpas en la playa,sorpresa
6,"Yo siempre compro por internet pero cada vez que hay Cyber Monday termino decepcionado, es mas la publicidad que reales ofertas",enojo
7,! Arjonaaaa !!!!! !Este otro ano vas fijo !!!!,sorpresa
8,!Partidazo!,alegria
9,"Medicion de pobreza, mas moderna y transparente: En enero de 2014 la Comision para la Medicion de la Pobreza, planteo un conjunto de propuestas de actualizacion de la metodologia de medicion de pobreza por ingresos, asi como la incorporacion de un enfoque de medicion multidimensional Sus sugerencias fueron recibidas por el gobierno de la Presidenta Bachelet y analizadas en detalle durante 2014 por una Mesa Tecnica Interinstitucional, conformada por el Ministerio de Desarrollo Social (MDS) y el Instituto Nacional de Estadisticas (INE), con el apoyo tecnico de Comision Economica para America Latina (CEPAL) En el ambito de la pobreza multidimensional, se conto ademas con la asesoria permanente de la Iniciativa para la Pobreza y el Desarrollo Humano de la Universidad de Oxford (OPHI, por su sigla en ingles)",sorpresa


## Ejercicio 1: Entender el efecto de cada técnica de normalización de texto

Las técnicas de procesamiento de texto, cuando tienen que realizar análisis del texto leído, deben buscar la simplificación del texto para poder trabajar sobre un universo de términos más simple y normalizado. Esto puede considerar eliminar tildes, evitar palabras comunes, usar la raíz de múltiples términos, entre otros.

A continuación se aplican estas diferentes técnicas, permitiendo ver el efecto (potencialmente positivo) de cada una de ellas.

**Pregunta 1 (2 puntos)**: ¿cuál de todas estas técnicas es la que logra el mayor efecto positivo en los modelos de análisis de sentimiento de la siguiente sección?

La idea es eliminar (comentar la línea de código respectivo) cada instrucción de normalización y comparar cómo afecta en el accuracy de los modelos más abajo (se puede enfocar sólo en el Árbol, que toma menos tiempo de ejecución, para ver su desempeño según estos cambios), finalmente determinando cuál es la que tiene mayor efecto. Aquella técnica que tiene más efecto es la que provoca que - al ser eliminada - el accuracy del o los modelos implementados más adelante empeore más. Se pide incluir una tabla de ejecuciones comparadas, que sustente la respuesta.


In [ ]:
library(tm)
library(SnowballC)

# Construye el Corpus: el universo de texto que se usará para entrenar los modelos.
corpus.original <- Corpus(VectorSource(commentsdataset$texto))

# Se selecciona y muestra (sin normalizar) un comentario de ejemplo aleatorio dentro del corpus
random_index <- floor(runif(1, min=0, max=length(corpus.original)))
content(corpus.original[[random_index]])

################################
# NORMALIZACIÓN DEL TEXTO
# EJERCICIO: probar eliminando (comentando anteponiendo #) una por una estas acciones,
# para ver cuál tiene mayor efecto en la calidad del modelo de análisis
################################
# Se saca una copia ('corpus') de trabajo, para no alterar el original
corpus <- corpus.original

# Se pasan todas las palabras a minúsculas
corpus <- tm_map(corpus, tolower)
# Se eliminan todos los signos de puntuación
corpus <- tm_map(corpus, removePunctuation)
# Se eliminan todos los números
corpus <- tm_map(corpus, removeNumbers)
# Se eliminan las stop words (palabras comunes, irrelevantes)
corpus <- tm_map(corpus, removeWords, c(stopwords("spanish")))
# Se lleva cada palabra a su raíz (stemming)
corpus <- tm_map(corpus, stemDocument)

# Se muestra el mismo ejemplo aleatorio, pero en texto normalizado
content(corpus[[random_index]])


Loading required package: NLP



[1] "ESPERA JAJAJA ESTA HA SIDO BUENA JAJJAJA  ENSERIO"

Warning message in tm_map.SimpleCorpus(corpus, tolower):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, removePunctuation):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, removeNumbers):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, removeWords, c(stopwords("spanish"))):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, stemDocument):
“transformation drops documents”


[1] "espera jajaja buena jajjaja enserio"

## Ejercicio 2: Construcción de un Vocabulario con Términos Significativos

Los clasificadores reciben un X de entrada de una dimensión fija. Por lo tanto los X de este ejemplo de análisis de texto, comentarios de cantidad variable de palabras, no se pueden usar tal cual vienen.

Por ello, la siguiente porción de código transforma los comentarios de texto variable en un vector de ocurrencia de palabras desde un vocabulario, el cual se construye referenciando todas las palabras distintas (ya normalizadas) del Corpus. Esto se traduce en que un comentario que tiene la expresión ".. resultados excelentes ... ", tendría una intersección con el comentario "... excelente como resultó ..." y por ello podrían ser interpretadas en forma equivalente.

Pero el desafío de esta vectorización en base a un vocabulario es la cantidad de dimensiones. Un vocabulario perfectamente puede tener varios miles de palabras diferentes (ver resultado en 2.A), entonces la dimensión del vector X es de esos varios miles.

Esto motiva a reducir la dimensionalidad del problema (el tamaño del vocabulario) al reconocer cuáles son los términos más relevantes. Esto se hace con removeSparseTerm() y un umbral alto (0.995 reduce la gran cantidad de términos que aparecen tienen al menos un 99.5% de 0s en la columna), como se ve en en 2.B. Mientras mayor es el número, mayor cantidad de términos ocasionales o esporádicos se conservan. En otras palabras, mientras menor es el número, mayor es la exigencia para un término de ser considerado valioso y quedar como parte del dataset. (https://www.rdocumentation.org/packages/tm/versions/0.7-8/topics/removeSparseTerms)

**Pregunta 2.1 (1 punto)** ¿cuál es el valor usado en removeSparseTerm() para lograr mejores resultados en los modelos de clasificación?

**Pregunta 2.2 (1 punto)** ¿cómo se interpreta el mejor/peor resultado con más o menos términos considerados (o columnas del dataset)?

Para ambas preguntas pueden elegir evaluar los resultados del Árbol de Decisión solamente, o también considerar la Red Neuronal (que toma mucho tiempo de ejecución; por eso se deja su ejecución opcional)


In [ ]:
#######################################################
# Indexación de términos: creación de un Vocabulario
#######################################################

# Primero una matriz de ocurrencia de términos o palabras (DTM: Document-term matrix).
# Las filas son los comentarios y las columnas son las palabras diferentes encontradas (varios miles).
termMatrix <- DocumentTermMatrix(corpus)
dim(termMatrix)   # Resultado 2.A -> Las columnas son todos los términos diferentes encontrados en corpus

# Entonces, se eliminan las palabras menos relevantes (sparse terms: términos dispersos)
# lo que resulta en una reducción dimensional (potencialmente grande)
# Ejercicio 2.1: ¿qué factor de eliminación (en rango 0.99x) da mejores resultados? Probar cambiando x entre 0 y 9.
# Ejercicio 2.2: ¿qué implica o cómo se interpreta ese cambio de valor y su efecto en el dataset?
termMatrixLight <- removeSparseTerms(termMatrix, 0.997)
dim(termMatrixLight)  # Resultado 2.B -> Se puede ver que se reduce significativamente la cantidad de columnas (palabras)

# Re-formatea como un DataFrame
corpus.procesado <- as.data.frame(as.matrix(termMatrixLight))

# Se construye el dataset para entrenamiento, recuperando el comentario original, sin procesar
# agregando la columna "emoción"
corpus.procesado$emoción <- as.factor(commentsdataset$emoción)

# Se muestra el vocabulario y su tamaño
dim(corpus.procesado)

[1]  6901 16848

[1] 6901  370

[1] 6901  371

## Paso 3: Ejecución modelos de predicción según conjuntos de entrenamiento y validación

En este caso, el dataset de ejemplos etiquetados se divide en dos (Hold-out) para entrenar y validar con conjuntos disjuntos.

No se necesita modificar esta sección, aunque es válido y potencialmente interesante revisar diferentes valores de proporción entrenamiento/validación.


In [ ]:
library(caTools)

################################################################
# Versión simple para crear conjuntos de entrenamiento y de test.
# Se puede dejar la proporción original de 0.70,
# pero se puede cambiar si se piensa que puede hacer una diferencia.
ratio <- sample(nrow(corpus.procesado),nrow(corpus.procesado)*0.70)
training.set = corpus.procesado[ratio,]
testing.set  = corpus.procesado[-ratio,]

cat("Dimensiones Entrenamiento/Test")
dim(training.set)
dim(testing.set)

head(testing.set, 10)

Dimensiones Entrenamiento/Test

[1] 4830  371

[1] 2071  371

,favor,cuanto,premio,chile,dio,hace,part,sera,tambien,cada,⋯,pide,onu,agua,biobio,jajajajaja,sube,cae,crisi,gustado,emoción
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,enojo
4,0,0,0,1,1,1,1,1,1,0,⋯,0,0,0,0,0,0,0,0,0,enojo
7,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,sorpresa
16,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,enojo
17,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,enojo
24,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,enojo
26,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,alegria
28,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,enojo
30,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,sorpresa


## Ejercicio 3: Interpretación de desempeño de modelos de clasificación de referencia

Habiendo definido y establecido los conjuntos de entrenamiento y de test, a continuación se ejecutan dos diferentes modelos de clasificación: Árbol de Decisión y más abajo una Red Neuronal (NNET). Cada uno obtiene su resultado, mostrando sus indicadores de desempeño.

Nótese que ninguno de los 2 modelos da resultados notoriamente buenos, lo cual se debe a la inherente complejidad de los casos de lenguaje natural y contar sólo con unos pocos miles de ejemplos (en total son cerca de 7.000 ejemplos, divididos en 3 clases, lo cual no alcanza a reflejar la real diversidad de palabras asociadas a cada una de las clases, cosa que se empieza a lograr desde las decenas de miles de ejemplos por clase).

Sin embargo, hay algunas observaciones que salen de los resultados de desempeño, además del hecho de que la red neuronal toma cerca de 10x el tiempo del Árbol de Decisión. De estas observaciones, se pide responder lo siguiente.

**Pregunta 3.1 (1 punto)** Se ve que hay un modelo con accuracy mejor que el otro, pero ¿cuál de los dos se puede considerar el mejor modelo al mirar su desempeño más completo y por qué?

**Pregunta 3.2 (1 punto)** Se puede observar que el Árbol de Decisión tiene un muy mal desempeño en reconocer una de esas clases. ¿Cuál y por qué podría darse esto? (En otras palabras, ¿qué característica de este modelo puede provocar que en este caso una de las clases no sea factible de predecir?


**Árbol de Decisión**

In [ ]:
library(caret)
library(rpart)

DT_model <- rpart(emoción ~ ., data=training.set, method="class", minbucket=20)

DT_predictTraining <- predict(DT_model, training.set, type = "class")
DT_predictTesting <- predict(DT_model, testing.set, type = "class")
cat("\n\n************* Resultados Árbol de Decisión - Testing *************\n")
confusionMatrix(DT_predictTesting, as.factor(testing.set$emoción))



************* Resultados Árbol de Decisión - Testing *************


Confusion Matrix and Statistics

          Reference
Prediction alegria enojo sorpresa
  alegria       22     0        0
  enojo        794   942      313
  sorpresa       0     0        0

Overall Statistics
                                          
               Accuracy : 0.4655          
                 95% CI : (0.4438, 0.4872)
    No Information Rate : 0.4549          
    P-Value [Acc > NIR] : 0.1714          
                                          
                  Kappa : 0.0206          
                                          
 Mcnemar's Test P-Value : NA              

Statistics by Class:

                     Class: alegria Class: enojo Class: sorpresa
Sensitivity                 0.02696      1.00000          0.0000
Specificity                 1.00000      0.01949          1.0000
Pos Pred Value              1.00000      0.45974             NaN
Neg Pred Value              0.61249      1.00000          0.8489
Prevalence                  0.39401      0.45485        

**Red Neuronal**

Nótese que toma varios minutos la ejecución de entrenamiento y evaluación de esta red, por lo que se recomienda hacer uso estratégico de los cambios en la sección/ejercicio 1 y 2 anteriores antes de probar.

In [ ]:
library(nnet)

# Parámetros de nnet():
#     size: moverse entre 5 y 60. Es la cantidad de nodos de la única capa intermedia.
#     maxit: 20 para pruebas cortas, hasta más de 100 (varios minutos de ejecución)
#     MaxNWts: cantidad tope de arcos de conexión. 20.000 se mantiene dentro del tope de RAM
cat("Entrenando Red Neuronal\n")
NN.model <- nnet(as.factor(emoción) ~ ., data=training.set, size=40, maxit=50, MaxNWts=20000)
NN.predict <- predict(NN.model, testing.set, type="class")

cat("Resultados Red Neuronal\n")
x <- testing.set[, 1:dim(corpus.procesado)[2]-1]
y <- testing.set[, dim(corpus.procesado)[2]]
y.predicted <- predict(NN.model, x, type = 'class')
confusionMatrix(as.factor(y.predicted), y)


Entrenando Red Neuronal
# weights:  7963
initial  value 6565.395637 
iter  10 value 4592.475193
iter  20 value 4426.971467
iter  30 value 4111.713377
iter  40 value 3583.990600
iter  50 value 3008.236600
final  value 3008.236600 
stopped after 50 iterations
Resultados Red Neuronal


Confusion Matrix and Statistics

          Reference
Prediction alegria enojo sorpresa
  alegria      489   397      136
  enojo        288   397      127
  sorpresa      84    91       62

Overall Statistics
                                          
               Accuracy : 0.4577          
                 95% CI : (0.4361, 0.4795)
    No Information Rate : 0.4273          
    P-Value [Acc > NIR] : 0.002804        
                                          
                  Kappa : 0.1101          
                                          
 Mcnemar's Test P-Value : 9.185e-08       

Statistics by Class:

                     Class: alegria Class: enojo Class: sorpresa
Sensitivity                  0.5679       0.4486         0.19077
Specificity                  0.5595       0.6501         0.89977
Pos Pred Value               0.4785       0.4889         0.26160
Neg Pred Value               0.6454       0.6124         0.85660
Prevalence                   0.4157       0.4273        